In [2]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time
# Load dataset
df, y = load_data()
#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.
print("df",df.head())
print()
print("data type",df.dtypes)
print()
print("Shape",df.shape)
print()
print("Null values",df.isnull().sum())
print()
print("discribe the data",df.describe())

<ipython-input-2-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-2-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


df         0         1         2         3         4         5         6    \
0  0.288585 -0.020294 -0.132905 -0.995279 -0.983111 -0.913526 -0.995112   
1  0.278419 -0.016411 -0.123520 -0.998245 -0.975300 -0.960322 -0.998807   
2  0.279653 -0.019467 -0.113462 -0.995380 -0.967187 -0.978944 -0.996520   
3  0.279174 -0.026201 -0.123283 -0.996091 -0.983403 -0.990675 -0.997099   
4  0.276629 -0.016570 -0.115362 -0.998139 -0.980817 -0.990482 -0.998321   

        7         8         9    ...       551       552       553       554  \
0 -0.983185 -0.923527 -0.934724  ... -0.074323 -0.298676 -0.710304 -0.112754   
1 -0.974914 -0.957686 -0.943068  ...  0.158075 -0.595051 -0.861499  0.053477   
2 -0.963668 -0.977469 -0.938692  ...  0.414503 -0.390748 -0.760104 -0.118559   
3 -0.982750 -0.989302 -0.938692  ...  0.404573 -0.117290 -0.482845 -0.036788   
4 -0.979672 -0.990441 -0.942469  ...  0.087753 -0.351471 -0.699205  0.123320   

        555       556       557       558       559       560  
0

In [4]:
y.values

array([[5],
       [5],
       [5],
       ...,
       [2],
       [2],
       [2]])

In [5]:
# Task 2: Encode class labels
# YOUR CODE HERE: Use LabelEncoder to encode class labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
encode_y = le.fit_transform(y.values.ravel())
print(encode_y)

[4 4 4 ... 1 1 1]


In [7]:
# Task 3: Scale the features using StandardScaler
# YOUR CODE HERE: Apply StandardScaler to df
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df)
print(scaled_df)

[[ 0.20064157 -0.0636826  -0.41962845 ... -0.68721921  0.40794614
  -0.00756789]
 [ 0.05594788  0.03148567 -0.25390836 ... -0.694138    0.40911698
   0.00787517]
 [ 0.07351535 -0.04341648 -0.07629468 ... -0.702239    0.4102883
   0.02650234]
 ...
 [-0.01566765  0.0167814   1.13222107 ... -0.56584847  0.64059683
   0.34870928]
 [ 0.21586648 -0.02812252 -0.86770988 ... -0.57766781  0.63147758
   0.29327564]
 [ 1.09620157  0.12919873 -1.67268082 ... -0.57392691  0.63274259
   0.33396081]]


In [9]:
# Task 4: Split the data into training and testing sets
# YOUR CODE HERE: Use train_test_split to split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_df, encode_y, test_size=0.2, random_state=1)

In [21]:
##TASK 5 - 1. Create a pipeline using Gaussian Naive Bayes
#          2. Fit the model to the training data
#          3. Predict values for test set
#          4. Print accuracy score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import GaussianNB
import time
start_time = time.time()
pipeline = Pipeline([
    ('classifier', GaussianNB())
])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
a_score = accuracy_score(y_test, y_pred)
print("Accuracy :",a_score)
c_report = classification_report(y_test, y_pred)
print("Classification Report :", c_report)
#TASK 6 - 1. Note the start time before defining the pipeline
#         2. Note the end time and report the difference as the time taken by the model training and inf
end_time = time.time()
time_taken = end_time - start_time
print("Time taken :",time_taken)

Accuracy : 0.7307953772943576
Classification Report :               precision    recall  f1-score   support

           0       0.94      0.70      0.80       247
           1       0.69      0.91      0.78       230
           2       0.76      0.75      0.75       185
           3       0.52      0.65      0.58       254
           4       0.77      0.93      0.84       282
           5       0.91      0.46      0.61       273

    accuracy                           0.73      1471
   macro avg       0.76      0.73      0.73      1471
weighted avg       0.77      0.73      0.73      1471

Time taken : 0.08387064933776855


In [24]:
# TASK 7 - K-Means for dimensionality reduction
n_clusters = 50
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(scaled_df.T)
selected_features_indices = [np.random.choice(np.where(kmeans.labels_ == i)[0]) for i in range(n_clusters)]
selected_features = scaled_df[:, selected_features_indices]
print(selected_features)

[[ 1.75070248 -0.8700415  -0.84576709 ... -0.71840922  0.83592584
  -0.7953587 ]
 [ 0.37823231 -0.91161272 -0.83736443 ...  0.24525519  1.5570466
   0.13061414]
 [-0.12889875 -0.92085432 -0.8268538  ... -0.67367648  2.34039882
   1.15233574]
 ...
 [-1.03813467  1.13049623  0.77398378 ...  0.4560065  -0.5444463
   0.27787672]
 [-1.02518343  1.13361745  0.55595075 ...  0.23656391 -1.61361693
  -0.44397761]
 [-0.46742019  1.03595673  1.14297738 ...  0.15363336 -2.16406282
  -0.73192085]]
